## Running, Visualizing, and Analyzing Molecular Dynamics (MD) Simulations ##
In this notebook, you will use Python tools to play around with simulations of atoms. All of our simulations will be performed using LAMMPS, an open-source MD package developed at Sandia National Lab.

## What am I looking at right now?
This is a "Jupyter notebook", a tool for using Python interactively. For example, you can quickly work with data, run simulations, visualize data, etc. then see how your results change if you change a parameter in your workflow. You hit `shift-enter` or `shift-return` to execute the code in a "cell". 

**Beware:**
- the good thing about notebooks is that they let you interact with your data in very flexible ways
- the bad thing is that you can execute cells out of order and overwrite variables in ways you forget or didn't expect

If you're getting weird results, it's best to either do `Cell->Run All` at the top to reset the entire notebook, or if really needed, `Kernel->Restart`

Let us run our first cell (right below) by hitting `shift-enter` inside the cell. This cell will import Python modules that we will need for the rest of the exercise. Note that the left of the cell will have a "star" marker (i.e., "In \[*\]") when it is running and an number (i.e., "In \[1\]") when it is done.

In [131]:
# Click inside of this cell and then hit `shift-enter` or `shift-return` to run the code.
# Time to set up the notebook! This might take a few minutes to finish.
# We are installing modules/libraries that other folks have created
# These libraries contain functions that will use (so that we don't reinvent the wheel)
import sys
old_stdout = sys.stdout
sys.stdout = open('stdout.txt', 'w')
!pip install lammps ase nglview mdtraj
# We now need to import or "load" the things we just installed.
import numpy as np
import mdtraj as md
import nglview as ngl
from lammps import lammps
from ase.io import read
import sys
from data.data import LJ_PARAMS, ATOMIC_MASS, CRYSTAL_STRUCTURE

# We also need to set up the simulation. 
# This defines all the rules for running the simulation except the element and temperature.
def simulate(element, temperature, size=3, crystallize=False):
    if crystallize:
        lattice_parameter = LJ_PARAMS[element]['cutoff'] * 0.8
    else:
        lattice_parameter = CRYSTAL_STRUCTURE[element]['lattice_param']
    old_stdout = sys.stdout
    sys.stdout = open('stdout.txt', 'w')
    l = lammps()
    setup = f"""
    clear
    units metal
    dimension 3
    boundary p p p
    atom_style atomic
    variable writefreq equal 100
    variable dt equal 0.001
    variable T equal {float(temperature) * 10}
    variable size equal {int(size)}
    
    lattice {CRYSTAL_STRUCTURE[element]['symmetry']} {lattice_parameter}
    region whole block 0 ${{size}} 0 ${{size}} 0 ${{size}}
    create_box 1 whole
    lattice {CRYSTAL_STRUCTURE[element]['symmetry']} {lattice_parameter} orient x 1 0 0 orient y 0 1 0 orient z 0 0 1
    create_atoms 1 region whole
    
    pair_style lj/cut {LJ_PARAMS[element]['cutoff']}
    pair_coeff * * {LJ_PARAMS[element]['epsilon']} {LJ_PARAMS[element]['sigma']}
    mass 1 {ATOMIC_MASS[element]}
    
    timestep ${{dt}}
    velocity all create $T 90909 rot yes mom yes dist gaussian
    fix mynvt all nvt temp $T $T $(dt*1000)
    
    dump mydump all atom ${{writefreq}} atom.lammpstrj

    compute numatoms all count/type atom
    variable ke_kilojoules equal "ke * 1.602e-19 / atoms * 6.022e23"
    thermo_style custom step temp etotal ke pe v_ke_kilojoules
    thermo 1000
    
    write_data initial.data
    """
    l.commands_string(setup)
    l.command("run 10000")

    ### Visualizing the trajectory ###
    # You'll see in the output of the previous block that a bunch of text was printed out. This is the "log" of the simulation and contains useful information. For our purposes, we can skip this and instead visualize the actual trajectory.
    # We will use `nglview` to display the trajectory. Once it loads, you can click play to see the atoms in motion!
    atoms = read("initial.data", format="lammps-data")
    atoms.write("initial.gro", format="gromacs")
    traj = md.load("atom.lammpstrj", top="initial.gro")
    view = ngl.show_mdtraj(traj)
    view.clear_representations()
    view.add_spacefill(selection="all", color="cyan", radius=0.05)
    view.stage.set_parameters(**{
        "clipNear": 0, "clipFar": 100, "clipDist": 1,
    })
    view.center()
    sys.stdout = old_stdout
    return view

### Running the simulation ###

Let us run a simulation of 10000 steps to see what happens.

NOTE: This next cell make take a few minutes to run. You will know it is still busy if you see the star in "In \[*\]

Once the first cell finishes running, run the rest of the simulations in the other cells.

In [132]:
element = "Au"
temperature = 300.0
view = simulate(element, temperature)
view

NGLWidget(max_frame=100)

In [133]:
element = "Au"
temperature = 2000.0
view = simulate(element, temperature)
view

NGLWidget(max_frame=100)

In [114]:
element = "Li"
temperature = 300.0
view = simulate(element, temperature)
view

NGLWidget(max_frame=100)

In [115]:
element = "Li"
temperature = 2000.0
view = simulate(element, temperature)
view

NGLWidget(max_frame=100)

In [129]:
element = "W"
temperature = 300.0
view = simulate(element, temperature)
view

NGLWidget(max_frame=100)

In [116]:
element = "W"
temperature = 2000.0
view = simulate(element, temperature)
view

NGLWidget(max_frame=100)

### Now you try it! ###

Select an element - Au for gold, Li for lithium, or W for tungsten and enter it in the cell below. Be sure to put quotes around the element symbol. Pick any temperature you want and enter it in the cell below also.

In [132]:
element = 
temperature = 
view = simulate(element, temperature)
view

NGLWidget(max_frame=100)

## Crystallization

In [117]:
temperature = 400  # Kelvin
element = "Au"
view = simulate(element, temperature, crystallize=True)
view

NGLWidget(max_frame=100)

## Observation debrief
